In [1]:
import pandas as pd
import requests

In [2]:
app_token = "Rl5BUiRawpr4H2LA9OQeKB47L"

# Data Preprocessing

In [3]:
zillow_df = pd.read_csv("data/zillow_rent_data.csv")

In [10]:
complaints_data = requests.get(url="https://data.cityofnewyork.us/resource/erm2-nwe9.json",
                               params={"$$app_token": app_token, "$where": "created_date between '2018-10-01T00:00:00' and '2018-10-10T23:59:59.9999'", "$limit": 99999999}).json()
complaints_df = pd.DataFrame(complaints_data)

In [24]:
from datetime import datetime, timedelta

def generate_month_ranges(start_date, end_date):
    current = start_date
    print("initial: {}".format(current))
    temp = 0
    while current < end_date:
        temp += 1
        month_end = current.replace(day=1) + timedelta(days=32)
        month_end = month_end.replace(day=1) - timedelta(days=1) # 9 30
        yield (current, month_end)
        current = month_end + timedelta(days=1)
        if temp == 100:
            break

# 设置起始和结束日期
start_date = datetime(2018, 10, 1)
end_date = datetime(2023, 9, 30)

# 生成月份范围
month_ranges = list(generate_month_ranges(start_date, end_date))

initial: 2018-10-01 00:00:00


In [25]:
import os

for start, end in month_ranges:
    year = start.year
    csv_file = f"data/complaints_data_{year}.csv"
    print(f"Downloading data for the period: {start.strftime('%Y-%m-%d')} to {end.strftime('%Y-%m-%d')}")

    query = f"created_date between '{start.strftime('%Y-%m-%dT%H:%M:%S')}' and '{end.strftime('%Y-%m-%dT%H:%M:%S')}'"
    response = requests.get(
        url="https://data.cityofnewyork.us/resource/erm2-nwe9.json",
        params={"$$app_token": app_token, "$where": query, "$limit": 999999,
                "$select": "unique_key, created_date, closed_date, agency,  complaint_type, descriptor, location_type, incident_zip, latitude, longitude, borough"
                }
    )
    data = response.json()
    batch_df = pd.DataFrame(data)

    # 根据年份写入或追加到对应的CSV文件
    mode = 'a' if os.path.exists(csv_file) else 'w'
    batch_df.to_csv(csv_file, mode=mode, index=False, header=not os.path.exists(csv_file))

print("Data download complete.")

Data download complete.


In [26]:
complaints_2021_df = pd.read_csv("data/complaints_data_2021.csv")

C:\Users\dota\AppData\Local\Temp\ipykernel_11908\3808333257.py:1: DtypeWarning: Columns (7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  complaints_2021_df = pd.read_csv("data/complaints_data_2021.csv")


In [27]:
complaints_2021_df.shape

(3115045, 11)

In [13]:
complaints_df.columns

Index(['unique_key', 'created_date', 'closed_date', 'agency', 'agency_name',
       'complaint_type', 'descriptor', 'location_type', 'incident_zip',
       'incident_address', 'street_name', 'address_type', 'city',
       'facility_type', 'status', 'due_date', 'resolution_description',
       'resolution_action_updated_date', 'community_board', 'borough',
       'x_coordinate_state_plane', 'y_coordinate_state_plane',
       'open_data_channel_type', 'park_facility_name', 'park_borough',
       'latitude', 'longitude', 'location', 'cross_street_1', 'cross_street_2',
       'bbl', 'intersection_street_1', 'intersection_street_2',
       'taxi_pick_up_location', 'taxi_company_borough', 'bridge_highway_name',
       'bridge_highway_direction', 'road_ramp', 'bridge_highway_segment',
       'landmark', 'vehicle_type'],
      dtype='object')

In [16]:
complaints_df.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,intersection_street_1,intersection_street_2,taxi_pick_up_location,taxi_company_borough,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,landmark,vehicle_type
0,40517253,2018-10-11T00:00:00.000,2018-10-17T00:00:00.000,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,Other (Explain Below),10451,223 EAST 144 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40524495,2018-10-11T00:00:00.000,2018-10-12T00:00:00.000,DOHMH,Department of Health and Mental Hygiene,Unsanitary Animal Pvt Property,Cat,1-2 Family Dwelling,11367,144-46 69 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,40524470,2018-10-11T00:00:00.000,2018-10-18T00:00:00.000,DOHMH,Department of Health and Mental Hygiene,Rodent,Condition Attracting Rodents,1-2 Family Dwelling,11362,52-20 CONCORD STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40524469,2018-10-11T00:00:00.000,2018-10-19T15:57:58.000,DOHMH,Department of Health and Mental Hygiene,Rodent,Condition Attracting Rodents,3+ Family Mixed Use Building,10003,67 ST MARKS PLACE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,40524454,2018-10-11T00:00:00.000,2018-10-15T00:00:00.000,DOHMH,Department of Health and Mental Hygiene,Rodent,Rat Sighting,Other (Explain Below),11206,ARION PLACE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
tree_data = requests.get(url="https://data.cityofnewyork.us/resource/5rq2-4hqu.json",
                         params={"$$app_token": app_token, "$limit": 99999999999999999999}).json()
tree_df = pd.DataFrame(tree_data)

In [14]:
tree_df.shape

(683788, 42)

In [15]:
tree_df.head()

,created_at,tree_id,block_id,the_geom,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,...,st_assem,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp
0,08/27/2015,180683,348711,"{'type': 'Point', 'coordinates': [-73.84421521...",3,0,OnCurb,Alive,Fair,Acer rubrum,...,28,16,QN17,Forest Hills,4073900,New York,40.72309177,-73.84421522,1027431.14821,202756.768749
1,09/03/2015,200540,315986,"{'type': 'Point', 'coordinates': [-73.81867945...",21,0,OnCurb,Alive,Fair,Quercus palustris,...,27,11,QN49,Whitestone,4097300,New York,40.79411067,-73.81867946,1034455.70109,228644.837379
2,09/05/2015,204026,218365,"{'type': 'Point', 'coordinates': [-73.93660770...",3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,50,18,BK90,East Williamsburg,3044900,New York,40.71758074,-73.9366077,1001822.83131,200716.891267
3,09/05/2015,204337,217969,"{'type': 'Point', 'coordinates': [-73.93445615...",10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,53,18,BK90,East Williamsburg,3044900,New York,40.71353749,-73.93445616,1002420.35833,199244.253136
4,08/30/2015,189565,223043,"{'type': 'Point', 'coordinates': [-73.97597938...",21,0,OnCurb,Alive,Good,Tilia americana,...,44,21,BK37,Park Slope-Gowanus,3016500,New York,40.66677776,-73.97597938,990913.775046,182202.425999


To save time, I will save the datasets downloaded locally.

In [17]:
complaints_df.to_csv("data/complaints.csv")
tree_df.to_csv("data/tree.csv")


In [35]:
complaints_df = pd.read_csv("data/complaints.csv", index_col=0)
tree_df = pd.read_csv("data/tree.csv", index_col=0)

C:\Users\dota\AppData\Local\Temp\ipykernel_16268\96994190.py:1: DtypeWarning: Columns (8,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  complaints_df = pd.read_csv("data/complaints.csv", index_col=0)


## Data Cleaning & Data Filtering

In [36]:
zillow_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2015-01-31,...,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30
0,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,1606.206406,...,1994.653463,2027.438438,2042.237444,2049.325559,2016.531345,2023.438976,2031.558202,2046.144009,2053.486247,2055.771355
1,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,1257.814660,...,1749.697900,1738.217986,1747.305840,1758.407295,1758.891075,1762.980879,1771.751591,1779.338402,1795.384582,1799.631140
2,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,NaN,...,1701.217520,1706.900064,1706.067787,1723.722320,1735.484670,1752.132904,1756.990323,1754.429516,1757.602011,1755.031490
3,93144,6,79936,zip,TX,TX,El Paso,"El Paso, TX",El Paso County,NaN,...,1419.480272,1458.063897,1471.726681,1466.734658,1456.175660,1462.478506,1466.267391,1490.237063,1488.180414,1494.366097
4,62093,7,11385,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,NaN,...,2935.808220,2895.699421,2873.209025,2881.906361,2913.546218,2963.964134,3005.735342,3034.413822,3064.476503,3079.585783


In [37]:
tree_df.head()

,created_at,tree_id,block_id,the_geom,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,...,st_assem,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp
0,08/27/2015,180683,348711,"{'type': 'Point', 'coordinates': [-73.84421521...",3,0,OnCurb,Alive,Fair,Acer rubrum,...,28,16,QN17,Forest Hills,4073900,New York,40.723092,-73.844215,1.027431e+06,202756.768749
1,09/03/2015,200540,315986,"{'type': 'Point', 'coordinates': [-73.81867945...",21,0,OnCurb,Alive,Fair,Quercus palustris,...,27,11,QN49,Whitestone,4097300,New York,40.794111,-73.818679,1.034456e+06,228644.837379
2,09/05/2015,204026,218365,"{'type': 'Point', 'coordinates': [-73.93660770...",3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,50,18,BK90,East Williamsburg,3044900,New York,40.717581,-73.936608,1.001823e+06,200716.891267
3,09/05/2015,204337,217969,"{'type': 'Point', 'coordinates': [-73.93445615...",10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,53,18,BK90,East Williamsburg,3044900,New York,40.713537,-73.934456,1.002420e+06,199244.253136
4,08/30/2015,189565,223043,"{'type': 'Point', 'coordinates': [-73.97597938...",21,0,OnCurb,Alive,Good,Tilia americana,...,44,21,BK37,Park Slope-Gowanus,3016500,New York,40.666778,-73.975979,9.909138e+05,182202.425999


In [38]:
complaints_df.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,incident_zip,intersection_street_1,intersection_street_2,...,due_date,bbl,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,taxi_company_borough,vehicle_type,landmark
0,29831247,2015-01-31T23:59:46.000,2015-02-05T10:00:00.000,DOT,Department of Transportation,Street Condition,Pothole,10451,EAST 149 STREET,PARK AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29835781,2015-01-31T23:59:25.000,2015-02-01T02:50:39.000,NYPD,New York City Police Department,Noise - Residential,Loud Television,11375,NaN,NaN,...,2015-02-01T07:59:25.000,4.021860e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,29837850,2015-01-31T23:59:08.000,2015-04-02T06:33:30.000,DOHMH,Department of Health and Mental Hygiene,Food Establishment,Food Worker Hygiene,10016,NaN,NaN,...,2015-04-01T23:59:08.000,1.008820e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,29833911,2015-01-31T23:58:50.000,2015-02-01T01:28:12.000,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,10474,NaN,NaN,...,2015-02-01T07:58:50.000,2.027630e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,29837016,2015-01-31T23:58:47.000,2015-02-01T05:26:00.000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,10034,NaN,NaN,...,2015-02-01T07:58:47.000,1.022170e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Storing Data

In [3]:
!createdb FINAL_PROJECT
!psql --dbname FINAL_PROJECT -c 'CREATE EXTENSION postgis;'

'createdb' is not recognized as an internal or external command,
operable program or batch file.
'psql' is not recognized as an internal or external command,
operable program or batch file.


In [21]:
import psycopg2

In [22]:
conn = psycopg2.connect("dbname=final_project user=postgres password=123456")
cur = conn.cursor()

In [23]:
with open('schema.sql', 'w') as f:
    command = """
        CREATE TABLE zip_codes (
            zip_code Integer PRIMARY KEY,
            neighborhood TEXT,
            borough TEXT,
            geometry GEOMETRY(Point, 4326)
        );

        CREATE TABLE complaints (
            complaint_id Integer PRIMARY KEY,
            created_date TIMESTAMP,
            complaint_type TEXT,
            descriptor TEXT,
            zip_code Integer REFERENCES zip_codes(zip_code),
            latitude FLOAT,
            longitude FLOAT,
            geometry GEOMETRY(Point, 4326)
        );

        CREATE TABLE trees (
            tree_id Integer PRIMARY KEY,
            species TEXT,
            diameter_inches FLOAT,
            health TEXT,
            zip_code Integer REFERENCES zip_codes(zip_code),
            latitude FLOAT,
            longitude FLOAT,
            geometry GEOMETRY(Point, 4326)
        );

        CREATE TABLE average_rents (
            zip_code Integer PRIMARY KEY,
            year INT,
            avg_rent DECIMAL
        );
    """
    f.write(command)

In [24]:
cur.execute(command)
conn.commit()

# Understanding Data

# Visualizing Data